In [1]:
import geopandas as gpd
import pandas as pd
from src.ecodonut.preprocessing import merge_objs_by_buffer,project_points_into_polygons,min_max_normalization


def industrial_preprocessing(gdf_OVNE, gdf_industrial):
    gdf_OVNE.to_crs(3857, inplace=True)
    gdf_OVNE = gdf_OVNE[
        gdf_OVNE['name'].str.upper().str.contains(
            'ПОЛИГОН|ЗАВОД|ТБО|ТКО|ЦЕХ|КОТЕЛЬНАЯ|РУДНИК|КАНАЛ|КАРЬЕР|СТАНЦИЯ|ПРОИЗВОД|ПРОМЫШЛЕН')]
    gdf_OVNE = gdf_OVNE.filter(items=['name', 'geometry', 'dangerous_level'])
    
    gdf_industrial.to_crs(3857, inplace=True)
    gdf_industrial = gdf_industrial.filter(items=['name', 'geometry'])

    def remove_nulls(x):
        if isinstance(x, list):
            x = [item for item in x if pd.notnull(item)]
            if len(x) == 0:
                return None
            if len(x) == 1:
                return x[0]
        return x

    gdf_industrial = merge_objs_by_buffer(gdf_industrial, 10)
    gdf_industrial['name'] = gdf_industrial['name'].apply(remove_nulls)
    union = project_points_into_polygons(gdf_OVNE, gdf_industrial)
    union[['name', 'dangerous_level']] = union.apply(lambda row: pd.Series(
        max(list(zip(row['name_right'], row['dangerous_level'])), key=lambda x: (-x[1], len(str(x[0]))))), axis=1)

    union['name'] = union.apply(lambda row: row['name'] if pd.notna(row['name']) else (
        row.name_left[0] if len(row.name_left) == 1 else row.name_left), axis=1)
    union.dropna(subset='name', inplace=True)
    union.drop(columns=['name_left', 'name_right', 'index_right'], inplace=True)
    union['type'] = 'industrial'
    union['dangerous_level'] = union['dangerous_level'].fillna(4)
    union['initial_impact'] = union.apply(lambda x: -10 if x.dangerous_level == 1 else (
        -8 if x.dangerous_level == 2 else (-6 if x.dangerous_level == 3 else -4)), axis=1)
    union['fading'] = union.apply(lambda x: 0.8 if x.dangerous_level == 1 else (
        0.6 if x.dangerous_level == 2 else (0.4 if x.dangerous_level == 3 else 0.2)), axis=1)
    union['total_impact_radius'] = 1000 * union['initial_impact'] * union['fading']
    union = gpd.GeoDataFrame(union, crs=3857)
    union = union.loc[~union['geometry'].is_empty]
    union = union[~union['geometry'].duplicated()]
    return union

In [3]:
city = 'Omsk'
# azs = gpd.read_file(f'.\\{city}\\fuel.geojson')
# dumps = gpd.read_file(f'.\\{city}\\landfill.geojson')

ovne = gpd.read_file(f'.\\{city}\\ovne.geojson')
ovne.rename(columns={'Наименование объекта':'name','Категория \nобъекта НВОС':'dangerous_level'}, inplace=True)
industrial = gpd.read_file(f'.\\{city}\\industrial.geojson')

# roads_fed = gpd.read_file(f'.\\{city}\\fuel.geojson')
# roads_reg = gpd.read_file(f'.\\{city}\\fuel.geojson')
# zhd = gpd.read_file(f'.\\{city}\\fuel.geojson')
# 
# forest = gpd.read_file(f'.\\{city}\\wood.geojson')
# forest['source'] = False
# recreation = gpd.read_file(f'.\\{city}\\nature_reserve.geojson')
# recreation2 = gpd.read_file(f'.\\{city}\\recreation.geojson')
# water = gpd.read_file(f'.\\{city}\\water.geojson')



In [5]:
# industrial = industrial_preprocessing(ovne,industrial)
industrial.explore(column='total_impact_radius')

In [ ]:
azs = azs.filter(items=['name', 'geometry'])
azs.to_crs(3857, inplace=True)
azs['geometry'] = azs['geometry'].buffer(40)
azs['type'] = 'petrol station'
azs['initial_impact'] = -4
azs['total_impact_radius'] = -1000 * 4 * 0.15
azs = azs.loc[~azs['geometry'].is_empty]
azs = azs[~azs['geometry'].duplicated()]
azs.name = azs.name.fillna('Без названия')

In [ ]:
dumps.to_crs(3857, inplace=True)
dumps['initial_impact'] = -3
dumps = dumps.loc[~dumps['geometry'].is_empty]
dumps = dumps[~dumps['geometry'].duplicated()]
dumps['area'] = dumps.area
dumps['area_normalized'] = min_max_normalization(dumps['area'])
dumps['total_impact_radius'] = -1000 * 3 * 0.4 * dumps['area_normalized']
dumps.name = dumps.name.fillna('Без названия')

In [ ]:
roads_reg = roads_reg.filter(items=['name', 'geometry'])
roads_reg.to_crs(3857, inplace=True)
roads_reg = roads_reg[~roads_reg['geometry'].duplicated()]
roads_reg['geometry'] = roads_reg['geometry'].buffer(20)
roads_reg = roads_reg['geometry'].unary_union
roads_reg = gpd.GeoDataFrame([roads_reg], columns=['geometry'], crs=32636)
roads_reg['name'] = 'Дорога регионального назначения'
roads_reg['type'] = 'regional_road'
roads_reg['initial_impact'] = -2
roads_reg['total_impact_radius'] = -1000 * 2 * 0.1

roads_fed = roads_fed.filter(items=['name', 'geometry'])
roads_fed = roads_fed[~roads_fed['geometry'].duplicated()]
roads_fed['geometry'] = roads_fed['geometry'].buffer(40)
roads_fed = roads_fed['geometry'].unary_union
roads_fed = gpd.GeoDataFrame([roads_fed], columns=['geometry'], crs=32636)
roads_fed['name'] = 'Дорога федерального назначения'
roads_fed['type'] = 'federal_road'
roads_fed['initial_impact'] = -4
roads_fed['total_impact_radius'] = -1000 * 4 * 0.1

In [ ]:
ZHD_gdf = zhd
ZHD_gdf = ZHD_gdf.filter(items=['geometry'])
ZHD_gdf = ZHD_gdf[~ZHD_gdf['geometry'].duplicated()]
ZHD_gdf['geometry'] = ZHD_gdf['geometry'].buffer(30)
ZHD_gdf = ZHD_gdf['geometry'].unary_union
ZHD_gdf = gpd.GeoDataFrame(geometry=[ZHD_gdf], crs=32636)
ZHD_gdf['name'] = 'ЖД Пути'
ZHD_gdf['type'] = 'railway'
ZHD_gdf['initial_impact'] = -3
ZHD_gdf['total_impact_radius'] = -1000 * 3 * 0.1